In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../input/wine-quality/winequalityN.csv')

df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.skew()

In [ ]:
df.isnull().sum()

In [ ]:
df['fixed acidity'] = df['fixed acidity'].fillna(df['fixed acidity'].mean())
df['volatile acidity'] = df['volatile acidity'].fillna(df['volatile acidity'].mean())
df['citric acid'] = df['citric acid'].fillna(df['citric acid'].mode()[0])
df['residual sugar'] = df['residual sugar'].fillna(df['residual sugar'].mode()[0])
df['chlorides'] = df['chlorides'].fillna(df['chlorides'].mean())
df['pH'] = df['pH'].fillna(df['pH'].mean())
df['sulphates'] = df['sulphates'].fillna(df['sulphates'].mean())

In [ ]:
df.isnull().sum()

In [ ]:
df.hist(figsize=(15,15), layout=(4,4), bins=20)

In [ ]:
#checking outliers using box plot
df.plot(kind="box",subplots=True,layout=(5,5),figsize=(15,15))

**Observation** from histogram and box plot

1. “fixed.acidity” is a measure of inside liquid concentration. 
    The histogram a right-skewed distributed with some outliers located at right side. 
    The most frequent values are between 7-8.
2. “volatile.acidity” is measure of acidity above-surface of liquid. 
    The histogram is right-skewed distributed with some outliers located at right side.
    The most frequent values are between 0.4-0.6.
3. “citric.acid” is right-skewed distributed with some outliers located at very right side. 
    The most frequent values 0. It’s also interesting a lot of wine have citric.acid = 0
4. “residual.sugar” is right-skewed distributed
5. “chlorides” is right-skewed distributed 
    The most frequent values are between 0.05-0.1
6. “free.sulfur.dioxide” is right-skewed distributed
7. “sulphates” is a right-skewed distributed

In [ ]:
df.loc[df['citric acid']>1.1,'citric acid'] = np.mean(df["citric acid"])
df.loc[df['residual sugar']>60,'residual sugar'] = np.mean(df["residual sugar"])
df.loc[df['chlorides']>5.5,'chlorides'] = np.mean(df["chlorides"])
df.loc[df['free sulfur dioxide']>250,'free sulfur dioxide'] = np.mean(df["free sulfur dioxide"])
df.loc[df['total sulfur dioxide']>330,'total sulfur dioxide'] = np.mean(df["total sulfur dioxide"])
df.loc[df['density']>1.01,'density'] = np.mean(df["density"])

In [ ]:
plt.figure(figsize=[14,6])
sns.heatmap(df.corr(), annot = True,cmap = 'coolwarm')

**Observation**

1. total sulfur dioxide is highly correlated with free sulfur dioxide
2. fixed acidity is highly correlated with density and citric acid
3. alcohol is highly correlated with quality
4. citric acid is inversely correlated with pH and volatile acidity

In [ ]:
sns.countplot(x='quality', data=df)

In [ ]:
#get_dummies() function which is used for handling categorical columns, 
#in this dataset ‘Type’ feature contains two types Red and White,
#where Red consider as 0 and white considers as 1
df = pd.get_dummies(df,drop_first=True)
df

In [ ]:
# as we can see from the above table some wine quality of 6 and 5 is white and some as red
# considering the best quality value of wine is 7
df['Excellent quality'] = [ 1 if x>=7 else 0 for x in df.quality] 
df

In [ ]:
y = df['Excellent quality']
y

In [ ]:
y.value_counts()

In [ ]:
x = df.drop(['quality','Excellent quality'], axis =1)
x

In [ ]:
from imblearn.combine import SMOTEENN
sn = SMOTEENN(random_state=0)
sn.fit(x,y)
x,y=sn.fit_resample(x,y)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model1 = KNeighborsClassifier(n_neighbors=3)
model1.fit(x_train,y_train)
y_pred1 = model1.predict(x_test)

from sklearn.metrics import accuracy_score
print("Accuracy Score of KNN:",accuracy_score(y_test,y_pred1))



In [ ]:
from sklearn.svm import SVC
model2 = SVC()
model2.fit(x_train,y_train)
y_pred2 = model2.predict(x_test)

print("Accuracy Score of SVC:",accuracy_score(y_test,y_pred2))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model3 = DecisionTreeClassifier(criterion='entropy',random_state=7)
model3.fit(x_train,y_train)
y_pred3 = model3.predict(x_test)

print("Accuracy Score of decission tree:",accuracy_score(y_test,y_pred3))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model4 = RandomForestClassifier(random_state=1)
model4.fit(x_train, y_train)
y_pred4 = model4.predict(x_test)


print("Accuracy Score:",accuracy_score(y_test,y_pred4))

In [ ]:
submission = pd.DataFrame({'Real Values': y_test, 'Predicted Values': y_pred1})
submission.to_csv('wine_quality.csv',index = False)